In [23]:
import json
import pandas as pd
import hdbscan
import numpy as np
from TFTClusterer import TFTClusterer
from datetime import datetime, timedelta
from googledrivesave import trashfolder, outputtodrive
import pygsheets
from SQLGatherer import loaddb
from matplotlib import pyplot as plt
df = loaddb(hours=10)
gameversion = df['game_version'].max()
df=df.loc[df['game_version']==gameversion]

assert len(df) >= 100, "less than 100 matches in newest patch"
#Cluster Data

clusterclass=TFTClusterer(df)
#clusterclass.imputetraits()

In [24]:
units=clusterclass.unitsdf
units.head(5)

,character_id,items,name,rarity,tier,match_id,game_variation,participants.placement,participants.puuid
0,TFT3_TwistedFate,[],,0,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...
1,TFT3_Poppy,"[99, 12, 57]",,0,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...
2,TFT3_Zoe,[],,0,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...
3,TFT3_Ahri,[],,1,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...
4,TFT3_Annie,[6],,1,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...


In [28]:
allrecords = df.to_json(orient='records')

In [29]:
        units = pd.json_normalize(json.loads(allrecords), 
        record_path=['participants','units'],
        meta=['match_id','game_variation',['participants','placement'],['participants','puuid'],['participants','level']])


In [31]:
units['participants.level'].value_counts()

8    59066
9    52712
7    12442
6     2135
5     1821
4      187
3        3
Name: participants.level, dtype: int64

In [68]:
EXPECTED_PRICE = {
    1 : {
        3 : 10.4,
        4 : 10.4,
        5 : 10.4,
        6 : 10.4,
        7 : 10.4,
        8 : 10.4,
        9 : 10.4,},
    2 : {
        3 : 14.5,
        4 : 14.5,
        5 : 14.5,
        6 : 14.5,
        7 : 14.5,
        8 : 14.5,
        9 : 14.5,},
    3 : {
        3 : 96.8,
        4 : 32.8,
        5 : 20.0,
        6 : 16.8,
        7 : 15.4,
        8 : 14.5,
        9 : 14.5,},
    4 : {
        3 : 500,
        4 : 300,
        5 : 121, 
        6 : 36.8,
        7 : 24.8,
        8 : 18.8,
        9 : 12.8,},
    5 : {
        3 : 999,
        4 : 999,
        5 : 999,
        6 : 481,
        7 : 121,
        8 : 48.8,
        9 : 24.8,}
}

In [63]:
units['gold'] = units['rarity'] + 1
units.head(5)

,character_id,items,name,rarity,tier,match_id,game_variation,participants.placement,participants.puuid,participants.level,gold
0,TFT3_TwistedFate,[],,0,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...,7,1
1,TFT3_Poppy,"[99, 12, 57]",,0,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...,7,1
2,TFT3_Zoe,[],,0,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...,7,1
3,TFT3_Ahri,[],,1,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...,7,2
4,TFT3_Annie,[6],,1,2,RU_282184018,TFT3_GameVariation_MidGameFoN,6,ty5DB-AmC-iPdXtm7266F6cUmWOWKK_O0WLg0T0AGOtjc2...,7,2


In [70]:
units['ExpectedGold']=units.apply(
            lambda row:
            (3**((row['tier'] - 1))*1.15) * (row['gold'] + EXPECTED_PRICE[row['gold']][row['participants.level']]),
            axis = 1)

In [71]:
units['ExpectedGold'].describe()

count    128366.000000
mean         63.060462
std          39.957390
min          13.110000
25%          39.330000
50%          57.960000
75%          63.480000
max        1154.600000
Name: ExpectedGold, dtype: float64